In [1]:
import csv
import random
import nltk 
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import datetime
import time
import random
import itertools
import numpy as np
import pickle
from numpy.linalg import cholesky
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm.notebook
import sklearn
from sklearn.metrics import roc_auc_score


[nltk_data] Downloading package punkt to /home/expero/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

print(f'GPU available: {torch.cuda.is_available()}')

Sat Apr 22 23:48:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.01    Driver Version: 512.78       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| N/A   45C    P8    13W /  N/A |    525MiB /  8192MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# behaviour_train_file = '/content/drive/My Drive/Courses/NLP/MINDsmall_train/behaviors.tsv'
# news_train_file = '/content/drive/My Drive/Courses/NLP/MINDsmall_train/news.tsv'

# behaviour_dev_file = '/content/drive/My Drive/Courses/NLP/MINDsmall_dev/behaviors.tsv'
# news_dev_file = '/content/drive/My Drive/Courses/NLP/MINDsmall_dev/news.tsv'

behaviour_train_file = 'MINDsmall_train/behaviors.tsv'
news_train_file = 'MINDsmall_train/news.tsv'

behaviour_dev_file = 'MINDsmall_dev/behaviors.tsv'
news_dev_file = 'MINDsmall_dev/news.tsv'

In [5]:
def newsample(nnn,ratio):
    if ratio >len(nnn):
        return random.sample(nnn*(ratio//len(nnn)+1),ratio)
    else:
        return random.sample(nnn,ratio)

In [6]:
################# Reading news data 
# max_sentence_len = 0
sent_len_for_model = 30
with open(news_train_file) as news_f:
  news_k = news_f.readlines()

news_dict = {}
content ={}
word_dict_raw={'PADDING':[0,999999]}

i=0
for doc in news_k:
  d = doc.strip().split('\t')
  newsid = d[0]
  ## Format is id,category,sub-category,Title,Abstract,link,Wikidata/metadata
  if newsid not in news_dict:
    news_dict[newsid] = len(news_dict)
    tokenized_news = word_tokenize(d[3].lower())
    # max_sentence_len = max(max_sentence_len,len(tokenized_news))
    content[news_dict[newsid]] = [d[1].lower(),d[2].lower(),tokenized_news]
    for word in tokenized_news:
        if word in word_dict_raw:
            word_dict_raw[word][1]+=1
        else:
            word_dict_raw[word]=[len(word_dict_raw),1]

word_dict={}
for i in word_dict_raw:
    if word_dict_raw[i][1]>=2:
        word_dict[i]=[len(word_dict),word_dict_raw[i][1]]
print(len(word_dict),len(word_dict_raw))
##### words for each news
news_words=[]
news_index={}
for newsid in content:
    word_id=[]
    news_dict[newsid]=len(news_index)
    for word in content[newsid][2]:
        if word in word_dict:
            word_id.append(word_dict[word][0])
    word_id=word_id[:30]
    news_words.append(word_id+[0]*(30-len(word_id)))
news_words=np.array(news_words,dtype='int32')

20772 37536


In [7]:
cat_to_id = {}
cat_news_to_id = []
subcat_to_id = {}
subcat_news_to_id = []

for newsid in content: 
    cat = content[newsid][0]
    if cat not in cat_to_id:
        cat_to_id[cat] = len(cat_to_id)
    
    subcat = content[newsid][1]
    if subcat not in subcat_to_id:
        subcat_to_id[subcat] = len(subcat_to_id)

for newsid in content: 
    cat_news_to_id.append([cat_to_id[content[newsid][0]]])
    subcat_news_to_id.append([subcat_to_id[content[newsid][1]]])

cat_news_to_id=np.array(cat_news_to_id ,dtype='int32')
subcat_news_to_id=np.array(subcat_news_to_id ,dtype='int32')

In [8]:
def get_embedding(word_dict):
    embedding_dict={}
    cnt=0
    # with open('/data/wuch/glove.840B.300d.txt','rb')as f:
    # with open('/content/drive/My Drive/Courses/NLP/glove.840B.300d.txt','rb') as f:
    with open('glove.840B.300d.txt','rb') as f:
        linenb=0
        while True:
            line=f.readline()
            if len(line)==0:
                break
            line = line.split()
            word=line[0].decode()
            linenb+=1
            if len(word) != 0:
                vec=[float(x) for x in line[1:]]
                if word in word_dict:
                    embedding_dict[word]=vec
                    if cnt%1000==0:
                        print(cnt,linenb,word)
                    cnt+=1

    embedding_matrix=[0]*len(word_dict)
    cand=[]
    for i in embedding_dict:
        embedding_matrix[word_dict[i][0]]=np.array(embedding_dict[i],dtype='float32')
        cand.append(embedding_matrix[word_dict[i][0]])
    cand=np.array(cand,dtype='float32')
    mu=np.mean(cand, axis=0)
    Sigma=np.cov(cand.T)
    norm=np.random.multivariate_normal(mu, Sigma, 1)
    for i in range(len(embedding_matrix)):
        if type(embedding_matrix[i])==int:
            embedding_matrix[i]=np.reshape(norm, 300)
    embedding_matrix[0]=np.zeros(300,dtype='float32')
    embedding_matrix=np.array(embedding_matrix,dtype='float32')
    # print(embedding_matrix.shape)
    return embedding_matrix

In [9]:
embedding_mat=get_embedding(word_dict)

0 1 ,
1000 1326 situation
2000 2903 v.
3000 4611 solve
4000 6541 suicide
5000 8816 resolved
6000 11463 sticky
7000 14534 protesters
8000 18348 noises
9000 22934 qualifies
10000 28630 ordeal
11000 36112 amazes
12000 46352 nugget
13000 60191 charleston
14000 81502 shaw
15000 113411 williamsburg
16000 169878 re-opens
17000 284973 parr
18000 592101 hoda


Sampling the data

In [10]:
################# Reading click behavior data 
min_neg_samples_required = 10
npratio = 4
#########################
with open(behaviour_train_file) as f2:
  userdata_new = f2.readlines()

userid_dict_total = {}
cnt_gr = 0
cnt = 0 
############################################
all_train_id=[]
all_train_pn=[]    
all_label=[]
all_user_pos=[]

############################################

for user in userdata_new:
  cnt +=1
  line = user.strip().split('\t')
  userid = line[1]
  impre_id = line[0]
  if userid not in userid_dict_total:
    userid_dict_total[userid] = len(userid_dict_total)
  click_history = line[3].strip().split()
  new_line = line[4].strip().split()
  trainpos=[x.split('-')[0]  for x in new_line if x[-1]=='1']
  trainneg=[x.split('-')[0] for x in new_line if x[-1]=='0']
  
  if len(trainneg) > 0:
      for pos_sample in trainpos:
          pos_neg_sample = newsample(trainneg,npratio)
          pos_neg_sample.append(pos_sample)
          temp_label=[0 for i in range(npratio)]+[1]
          temp_id=list(range(npratio+1))
          random.shuffle(temp_id)

          shuffle_sample=[]
          shuffle_label=[]
          for id in temp_id:
              shuffle_sample.append(news_dict[pos_neg_sample[id]])
              shuffle_label.append(temp_label[id])
          posset=list(set(click_history)-set([pos_sample]))
          allpos=[news_dict[p] for p in random.sample(posset,min(50,len(posset)))[:50]]
          allpos+=[0]*(50-len(allpos))
          all_train_pn.append(shuffle_sample)
          all_label.append(shuffle_label)
          all_train_id.append(userid_dict_total[userid])
          all_user_pos.append(allpos)
############################################
all_train_pn_total=np.array(all_train_pn,dtype='int32')
all_label_total=np.array(all_label,dtype='int32')
all_train_id_total=np.array(all_train_id,dtype='int32')
all_user_pos_total=np.array(all_user_pos,dtype='int32')

In [11]:
def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best


def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)

In [12]:
def generate_batch_data_test(all_test_pn,all_label,all_test_id,batch_size):
    inputid = np.arange(len(all_label))
    y=all_label
    batches = [inputid[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            candidate = news_words[all_test_pn[i]]
            browsed_news=news_words[all_test_user_pos[i]]
            browsed_news_split=[browsed_news[:,k,:] for k in range(browsed_news.shape[1])]
            userid=np.expand_dims(all_test_id[i],axis=1)
            label=all_label[i]

            yield ([candidate]+ browsed_news_split+[userid], label)
    

In [13]:
print(all_train_pn_total.shape)
print(all_label_total.shape)
print(all_train_id_total.shape)
print(all_user_pos_total.shape)

(236344, 5)
(236344, 5)
(236344,)
(236344, 50)


In [14]:
unique_users,user_counts = np.unique(all_train_id_total,return_counts=True)
selected_user_ids = unique_users[user_counts>15]
########################
final_id_mask = np.isin(all_train_id_total,selected_user_ids)

final_pn_total= all_train_pn_total[final_id_mask,]
final_label_total = all_label_total[final_id_mask,]
final_id_total = all_train_id_total[final_id_mask]
final_user_pos_total = all_user_pos_total[final_id_mask,]
###################
unique_id,first_occurance = np.unique(np.flip(final_id_total),return_index=True)
final_index = len(final_id_total)-1-first_occurance
######################
all_index = np.array(range(0,len(final_id_total)))

test_index = np.isin(all_index,final_index)

train_index = ~test_index
##########################
all_test_cand_news = final_pn_total[test_index,]
all_test_cand_label = final_label_total[test_index,]
all_test_usr_ids = final_id_total[test_index]
all_test_browsed_news = final_user_pos_total[test_index,]
# all_test_index = np.array(range(0,len(all_test_usr_ids)))

all_train_cand_news = final_pn_total[train_index,]
all_train_cand_label = final_label_total[train_index,]
all_train_usr_ids = final_id_total[train_index]
all_train_browsed_news = final_user_pos_total[train_index,]



In [15]:
print(all_train_cand_news.shape)
print(all_train_cand_label.shape)
print(all_train_usr_ids.shape)
print(all_train_browsed_news.shape)

(54335, 5)
(54335, 5)
(54335,)
(54335, 50)


In [16]:
print(all_test_cand_news.shape)
print(all_test_cand_label.shape)
print(all_test_usr_ids.shape)
print(all_test_browsed_news.shape)

(2336, 5)
(2336, 5)
(2336,)
(2336, 50)


In [17]:
import torch
from torch import nn

class AttentiveMultiView(nn.Module):
    def __init__(self, embedding_mat, CAT_LEN, SUBCAT_LEN, DIM_CAT=50, DIM_EMB=300, DIM_CONV_EMB=400, DIM_ATTN=200, NUM_HIER=1, CONTEXT_WIN=3, MAX_SLEN=30):
        super().__init__()
        
        self.DIM_EMB = DIM_EMB
        self.DIM_CONV_EMB = DIM_CONV_EMB
        self.MAX_SLEN = MAX_SLEN
        self.DIM_ATTN = DIM_ATTN 

        self.cat_embed = nn.Embedding(CAT_LEN, DIM_CAT)
        self.subcat_embed = nn.Embedding(SUBCAT_LEN, DIM_CAT)
        self.all_c_sc_linear = nn.Linear(DIM_CAT, DIM_CONV_EMB) 

        self.embed = nn.Embedding.from_pretrained(torch.from_numpy(embedding_mat))
        self.dropout = nn.Dropout(p=0.2)
        self.conv1d = nn.Conv1d(DIM_EMB, DIM_CONV_EMB, kernel_size=CONTEXT_WIN, padding='same', stride=1)
        self.relu = nn.ReLU()
        self.attn_layer = nn.Linear(DIM_CONV_EMB, DIM_ATTN)
        self.attn_layer2 = nn.Linear(DIM_ATTN, 1)
        self.tanh = nn.Tanh()
        self.softmax = nn.LogSoftmax(dim=1)
        self.concat_layer = nn.Linear(DIM_ATTN*NUM_HIER, DIM_ATTN)

    def news_encoder(self, batch_news, batch_size, news_words, cat_news_to_id, subcat_news_to_id):
        
        # Categories
        cat_per_row = [cat_news_to_id[batch_news[i]] for i in range(len(batch_news))]
        cat_per_row = torch.stack(cat_per_row, axis=0).squeeze(-1)
        cat_emb = self.cat_embed(cat_per_row)
        r_c = self.all_c_sc_linear(cat_emb).unsqueeze(2)
        # print(r_c.shape)

        # Subcategories
        subcat_per_row = [subcat_news_to_id[batch_news[i]] for i in range(len(batch_news))]
        subcat_per_row = torch.stack(subcat_per_row, axis=0).squeeze(-1)
        subcat_emb = self.subcat_embed(subcat_per_row)
        r_sc = self.all_c_sc_linear(subcat_emb).unsqueeze(2)
        # print(r_sc.shape)

        # Passing News title
        news_per_row = [news_words[batch_news[i]] for i in range(len(batch_news))]
        news_per_row = torch.stack(news_per_row, axis=0)
        
        emb = self.dropout(self.embed(news_per_row))
        # TODO: need to check if -1 to put at dim 0 or dim 3
        emb = emb.reshape(-1, self.DIM_EMB, self.MAX_SLEN)
        context = self.dropout(self.relu(self.conv1d(emb)))
        context = context.reshape(batch_size, -1, self.MAX_SLEN, self.DIM_CONV_EMB)
        attn_t = self.tanh(self.attn_layer(context))
        attn_t = self.attn_layer2(attn_t).squeeze()
        attn_weights_t = self.softmax(attn_t)
        r_t = torch.bmm(context.reshape(-1, self.DIM_CONV_EMB, self.MAX_SLEN), attn_weights_t.reshape(-1, self.MAX_SLEN, 1)).reshape(batch_size, -1, self.DIM_CONV_EMB).unsqueeze(2)
        
        r_channel = torch.cat([r_c, r_sc, r_t], dim=2)
        r_channel = r_channel.reshape(-1, r_channel.shape[2], self.DIM_CONV_EMB)
        attn_v = self.tanh(self.attn_layer(r_channel))
        attn_v = self.attn_layer2(attn_v).squeeze()
        attn_weights_v = self.softmax(attn_v)
        output = torch.bmm(r_channel.reshape(-1, self.DIM_CONV_EMB, r_channel.shape[1]), attn_weights_v.reshape(-1, r_channel.shape[1], 1))

        return output # 500 x 400 x 1
    
    def forward(self, batch_cand_news, batch_cand_label, batch_browsed_news, news_words, cat_news_to_id, subcat_news_to_id):
        batch_size = batch_cand_news.shape[0]

        # Candidate News
        candNews_rep = self.news_encoder(batch_cand_news, batch_size, news_words, cat_news_to_id, subcat_news_to_id).reshape(batch_size, -1, self.DIM_CONV_EMB)
        # print(candNews_rep.shape)

        # Browsed/Clicked News
        browNews_rep = self.news_encoder(batch_browsed_news, batch_size, news_words, cat_news_to_id, subcat_news_to_id).reshape(batch_size, -1, self.DIM_CONV_EMB)
        attn_n = self.tanh(self.attn_layer(browNews_rep))
        attn_n = self.attn_layer2(attn_n).squeeze()
        attn_weights_n = self.softmax(attn_n)
        user_rep = torch.bmm(browNews_rep.reshape(browNews_rep.shape[0], self.DIM_CONV_EMB, -1), attn_weights_n.unsqueeze(2))
        # print(user_rep.shape)
        
        output = nn.functional.log_softmax(torch.bmm(candNews_rep,user_rep).squeeze(),dim=1)
        return output

In [18]:
# attentive_mv = AttentiveMultiView(embedding_mat, CAT_LEN = len(cat_to_id), SUBCAT_LEN = len(subcat_to_id), DIM_EMB=300, DIM_CONV_EMB=400, DIM_ATTN=200, NUM_HIER=1, CONTEXT_WIN=3)


In [19]:
# x = attentive_mv.forward(torch.LongTensor(all_test_cand_news[:100]), torch.LongTensor(all_test_cand_label[:100]), torch.LongTensor(all_test_browsed_news[:100]), news_words=torch.LongTensor(news_words), cat_news_to_id=torch.LongTensor(cat_news_to_id), subcat_news_to_id=torch.LongTensor(subcat_news_to_id))

In [20]:
def train(all_train_cand_news,all_train_cand_label,all_train_browsed_news,all_train_usr_ids,news_words,cat_news_to_id,subcat_news_to_id,model,nEpochs):
    #optimizer = optim.Adadelta(lstm.parameters(), lr=0.1)
    #TODO: initialize optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    batch_size = 16

    for epoch in range(nEpochs):
        totalLoss = 0.0
        for batch in tqdm.notebook.tqdm(range(0, len(all_train_usr_ids), batch_size), leave=False):
            model.zero_grad()
            #TODO: Implement gradient update.
            batch_cand_news = all_train_cand_news[batch:batch+batch_size,]
            batch_cand_label = all_train_cand_label[batch:batch+batch_size]
            # batch_id = all_train_id[batch:batch+batch_size]
            batch_browsed_news = all_train_browsed_news[batch:batch+batch_size,]


            logits = model(torch.LongTensor(batch_cand_news).cuda(), torch.LongTensor(batch_cand_label).cuda(), 
                              torch.LongTensor(batch_browsed_news).cuda(),torch.LongTensor(news_words).cuda(),
                              torch.LongTensor(cat_news_to_id).cuda(),torch.LongTensor(subcat_news_to_id).cuda())
            loss = nn.functional.cross_entropy(logits.to('cpu'), torch.FloatTensor(batch_cand_label))
            loss.backward()
            optimizer.step()
            totalLoss += loss.item()
            
        print(f"loss on epoch {epoch} = {totalLoss}")
        
        # batch_pn = all_test_pn
        # batch_label = all_test_label
        # batch_id = all_test_id
        # batch_user_pos = all_test_user_pos

        test_y_scores = []
        for batch in tqdm.notebook.tqdm(range(0, len(all_test_usr_ids), batch_size), leave=False):
            batch_cand_news = all_test_cand_news[batch:batch+batch_size,]
            batch_cand_label = all_test_cand_label[batch:batch+batch_size]
            # batch_id = all_train_id[batch:batch+batch_size]
            batch_browsed_news = all_test_browsed_news[batch:batch+batch_size,]
            logits = model(torch.LongTensor(batch_cand_news).cuda(), torch.LongTensor(batch_cand_label).cuda(), 
                                  torch.LongTensor(batch_browsed_news).cuda(),torch.LongTensor(news_words).cuda())
            y_score  = logits.detach().cpu().numpy()
            test_y_scores.append(y_score)
          
        test_y_scores = np.array(test_y_scores)
        test_y_scores = test_y_scores.reshape(-1, test_y_scores.shape[-1])
        mrr_sc = np.mean(np.array([mrr_score(all_test_cand_label[i],test_y_scores[i]) for i in range(len(test_y_scores))]))
        rocc_auc_score = np.mean(np.array([roc_auc_score(all_test_cand_label[i],test_y_scores[i]) for i in range(len(test_y_scores))]))
        ndcg_5_score = np.mean(np.array([ndcg_score(all_test_cand_label[i],test_y_scores[i],5) for i in range(len(test_y_scores))]))
        ndcg_10_score = np.mean(np.array([ndcg_score(all_test_cand_label[i],test_y_scores[i],10) for i in range(len(test_y_scores))]))
        print('MRR on test set: '+ str(mrr_sc))
        print('ROC-AUC on test set: '+ str(rocc_auc_score))
        print('NDCG@5 on test set: '+ str(ndcg_5_score))
        print('NDCG@10 on test set: '+ str(ndcg_10_score))
        print('----------------------------------------------------------------------------------------------------------')



In [21]:
attentive_mv = AttentiveMultiView(embedding_mat, CAT_LEN = len(cat_to_id), SUBCAT_LEN = len(subcat_to_id), DIM_EMB=300, DIM_CONV_EMB=400, DIM_ATTN=200, NUM_HIER=1, CONTEXT_WIN=3).cuda()


In [ ]:
train(all_train_cand_news,all_train_cand_label,all_train_browsed_news,all_train_usr_ids,news_words,cat_news_to_id,subcat_news_to_id,attentive_mv,5)

  0%|          | 0/3396 [00:00<?, ?it/s]